In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
plt.style.use('fivethirtyeight')


In [ ]:
df = pd.read_csv('TSLA.csv')
df.head()

In [ ]:
df = df.loc[:,['Date','Close']]
df.head()

In [ ]:
#plot the data
train = df.loc[df['Date'] <= '2021-12-31']
test = df.loc[df['Date'] > '2021-12-31']
test = test.set_index('Date')

test.index = pd.to_datetime(test.index)
test.head()

In [ ]:
#plotting
plt.figure(figsize=(12,6))
plt.plot(test['Close'], label='Price')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.show()

In [ ]:
#Simple Moving Average - Rolling Mean - We can do rolling mean of the number of days - k value
sma10 = df['Close'].rolling(10).mean()
sma2 = df['Close'].rolling(2).mean()
#The above code is for finding 10-days SMA.  This is done on the dataset

df['10-day SMA'] = np.round(sma10,decimals=3)
df['2-day SMA'] = np.round(sma2,decimals=3)
train = df.loc[df['Date'] <= '2021-12-31']
test = df.loc[df['Date'] > '2021-12-31']
test = test.set_index('Date')
test.index = pd.to_datetime(test.index)



plt.figure(figsize=(12,6))
plt.plot(test['Close'], label='Price')
plt.plot(test['10-day SMA'],label='10-day SMA')
plt.plot(test['2-day SMA'],label='2-day SMA')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.legend()
plt.show()


In [ ]:
df.head(15)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
df.tail(15)
df.head(15)

In [ ]:
#Laxmi look from here
#Finding the last date for the data available
#last_date = df['Date'].max()
#history = [x for x in df]
predictions = list()
#t = 3

for t in range(len(df)):
    #if (t!=0 && t!=1):
    #    print(df.loc[t+1]['Date'])
    #    print(df.loc[t+1]['2-day SMA'])
    if(t == 0 or t == 1):
        pred_value = np.NaN
    else:
        pred_value = df.loc[t-1]['2-day SMA']
    predictions.append(pred_value)
#Appending predictions to the dataset df
df.loc[:,'2-day Pred'] = predictions
#df.head(15)
df.tail(15)

In [ ]:
df_new = df   


In [ ]:
df_new.tail(10)

In [ ]:
start_date = pd.to_datetime(df['Date'].max())
#print(start_date)
#print(type(start_date))
end_date = start_date + pd.DateOffset(days=7)

#print(type(end_date))
date_list_pred = list()
while start_date <= end_date:
    next_date = start_date + pd.DateOffset(days=1)
    prev_date = start_date + pd.DateOffset(days=-1)
    if not (next_date.weekday()==5 or next_date.weekday()==6):
        #df_new.loc[len(df_new.index)] = [next_date,np.nan,np.nan,np.nan,
        #
        #print(df_new.loc[len(df_new.index)-1]['2-day Pred'])
        #print(df_new.loc[len(df_new.index)-2]['2-day Pred'])
        sma2_pred = (df_new.loc[len(df_new.index)-1]['2-day Pred'] + df_new.loc[len(df_new.index)-2]['2-day Pred'])/2
        df_new.loc[len(df_new.index)] = [next_date,np.nan,np.nan,np.nan,sma2_pred]
    start_date = start_date + pd.DateOffset(days=1)
    #df_new.loc[len(df_new.index)] = [next_date,np.nan,np.nan,np.nan,np.nan]    
df_new.tail(10)


In [ ]:
#Calculating the predictions for the next 5 days from current date based on the 2-day SMA
#predictions = df['2-day SMA'].rolling(2).mean()
#df['2-day SMA'].head(15)

X = df['Close'].values
size = np.sum(df['Date'] <= '2021-12-31')
train, test = X[0:size], X[size:len(X)]

history = [x for x in train]
predictions = list()

for t in range(len(test)) + 5:
  #Take the first value
  yhat = output[0]
  predictions.append(yhat)
  obs = test[t]
  history.append(obs)
  #history.append(yhat)


In [ ]:
prediction.head(15)

In [ ]:
#Weighted moving average - for 10 days
weights = np.arange(1,11)
weights
wma10 = df['Close'].rolling(10).apply(lambda price: np.dot(price,weights)/weights.sum(), raw = True)
wma10

In [ ]:
np.dot([3,3],[2,5])

In [ ]:
#Exponential moving average
ema10 = df['Close'].ewm(span=10,adjust= False).mean()
ema2 = df['Close'].ewm(span=2,adjust= False).mean()

df['10-day EMA'] = np.round(ema10,decimals=3)
df['2-day EMA'] = np.round(ema2,decimals=3)
train = df.loc[df['Date'] <= '2021-12-31']
test = df.loc[df['Date'] > '2021-12-31']
test = test.set_index('Date')
test.index = pd.to_datetime(test.index)



plt.figure(figsize=(12,6))
plt.plot(test['Close'], label='Price')
plt.plot(test['10-day EMA'],label='10-day EMA')
plt.plot(test['2-day EMA'],label='2-day EMA')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.legend()
plt.show()


In [ ]:
df.head(10)

In [ ]:
#ARIMA - Proper model which puts the differentiation into prediction
#How will be my model performance for the next 1 day
from statsmodels.tsa.arima_model import ARIMA
X = df['Close'].values
size = np.sum(df['Date'] <= '2021-12-31')
train, test = X[0:size], X[size:len(X)]

history = [x for x in train]
predictions = list()

for t in range(len(test)):
  model = ARIMA(history, order=(1,1,0)) #Order p,q,d
  model_fit = model.fit()
  output = model_fit.forecast()
  #Take the first value
  yhat = output[0]
  predictions.append(yhat)
  obs = test[t]
  history.append(obs)
  #history.append(yhat)

test = df.loc[df['Date'] > '2021-12-31']
test = test.set_index('Date')
test.index = pd.to_datetime(test.index)
test.loc[:,'ARIMA'] = predictions



plt.figure(figsize=(12,6))
plt.plot(test['Close'], label='Price')
plt.plot(test['ARIMA'],label='ARIMA')
#plt.plot(test['2-day EMA'],label='2-day EMA')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.legend()
plt.show()


    

In [ ]:
#ARIMA - Proper model which puts the differentiation into prediction
#How will be my model performance for the next 1 day
from statsmodels.tsa.arima_model import ARIMA
X = df['Close'].values
size = np.sum(df['Date'] <= '2021-12-31')
train, test = X[0:size], X[size:len(X)]

history = [x for x in train]
predictions = list()

for t in range(len(test)):
  model = ARIMA(history, order=(1,2,0)) #Order p,d,q
  #P - AR
  # D - max 2
  # Q - MA (Order)
  model_fit = model.fit()
  output = model_fit.forecast()
  #Take the first value
  yhat = output[0]
  predictions.append(yhat)
  obs = test[t]
  history.append(obs)
  #history.append(yhat)

test = df.loc[df['Date'] > '2021-12-31']
test = test.set_index('Date')
test.index = pd.to_datetime(test.index)
test.loc[:,'ARIMA'] = predictions



plt.figure(figsize=(12,6))
plt.plot(test['Close'], label='Price')
plt.plot(test['ARIMA'],label='ARIMA')
#plt.plot(test['2-day EMA'],label='2-day EMA')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.legend()
plt.show()


In [ ]:
!pip install 